In [ ]:
%pip install pybaseball==2.2.1
%matplotlib inline
import pandas as pd; 
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.float_format', lambda x: '%.5f' % x)
import seaborn as sns
import sklearn as sk
import pybaseball as pyb
from matplotlib import pyplot as plt
from pybaseball import cache
cache.enable()
import warnings; warnings.filterwarnings('ignore')


In [ ]:
pitch_df = pyb.team_pitching(2015, 2019)
pitch_2019 = pyb.team_pitching(2018, 2019)
pitch_df

In [ ]:
# creating our df for predicting a full 2020 season 

pitch_df_copy = pitch_df.copy()

pitch_df_copy = pitch_df_copy.loc[:, ['Season','Team', 'G', 'H', 'SO', 'ERA', 'H/9', 'HR/9', 'EV']]

pitch_df_copy['G_Next_Year'] = pitch_df_copy.sort_values(['Season','Team'], ascending=False).groupby('Team')['G'].shift()
pitch_df_copy['H_Next_Year'] = pitch_df_copy.sort_values(['Season','Team'], ascending=False).groupby('Team')['H'].shift()
pitch_df_copy['SO_Next_Year'] = pitch_df_copy.sort_values(['Season','Team'], ascending=False).groupby('Team')['SO'].shift()
pitch_df_copy['ERA_Next_Year'] = pitch_df_copy.sort_values(['Season','Team'], ascending=False).groupby('Team')['ERA'].shift()
pitch_df_copy['H/9_Next_Year'] = pitch_df_copy.sort_values(['Season','Team'], ascending=False).groupby('Team')['H/9'].shift()
pitch_df_copy['HR/9_Next_Year'] = pitch_df_copy.sort_values(['Season','Team'], ascending=False).groupby('Team')['HR/9'].shift()
pitch_df_copy['EV_Next_Year'] = pitch_df_copy.sort_values(['Season','Team'], ascending=False).groupby('Team')['EV'].shift()



pitch_df_copy = pitch_df_copy.loc[pitch_df_copy['G_Next_Year'].notnull()]
pitch_df_copy = pitch_df_copy.loc[pitch_df_copy['H_Next_Year'].notnull()]
pitch_df_copy = pitch_df_copy.loc[pitch_df_copy['SO_Next_Year'].notnull()]
pitch_df_copy = pitch_df_copy.loc[pitch_df_copy['ERA_Next_Year'].notnull()]
pitch_df_copy = pitch_df_copy.loc[pitch_df_copy['H/9_Next_Year'].notnull()]
pitch_df_copy = pitch_df_copy.loc[pitch_df_copy['HR/9_Next_Year'].notnull()]
pitch_df_copy = pitch_df_copy.loc[pitch_df_copy['EV_Next_Year'].notnull()]



pitch_df_copy


In [ ]:
pitch_df_copy.corr() 

In [ ]:
# splitting our data for x, y training and testing

from sklearn.model_selection import train_test_split

x = pitch_df_copy[['G', 'H', 'SO', 'ERA', 'H/9', 'HR/9', 'EV']].values
y= pitch_df_copy[['G_Next_Year', 'H_Next_Year', 'SO_Next_Year', 'ERA_Next_Year', 'H/9_Next_Year','HR/9_Next_Year', 'EV_Next_Year']].values

print('Original Data Shape - X: {0}, Y: {1}'.format(x.shape, y.shape))
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)       #test size is 20% of original


print('Train Data Shape - X{0}, Y:{1}.'.format(x_train.shape, y_train.shape) )
print('Test Data Shape - X{0}, Y:{1}.'.format(x_test.shape, y_test.shape) )

Original Data Shape - X: (120, 7), Y: (120, 7)
Train Data Shape - X(96, 7), Y:(96, 7).
Test Data Shape - X(24, 7), Y:(24, 7).


In [ ]:
#taking our training data and fitting it into our regression model 

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error


lr = LinearRegression()

lr.fit(x_train, y_train)

LinearRegression()

In [ ]:

y_pred = lr.predict(x_train)

print ('Mean number of hits:', x_train[:, 0].mean())
print ('Mean absolute error:', mean_absolute_error(y_pred, y_train))

In [ ]:
pitch_2019_copy = pitch_2019.copy()

pitch_2019_copy = pitch_2019_copy.loc[:,['Season','Team', 'G', 'H', 'SO', 'ERA', 'H/9', 'HR/9', 'EV']]

pitch_2019_copy['2019_actual_g'] = pitch_2019_copy.sort_values(['Team', 'Season'], ascending=False).groupby('Team')['G'].shift()
pitch_2019_copy['2019_actual_h'] = pitch_2019_copy.sort_values(['Team', 'Season'], ascending=False).groupby('Team')['H'].shift()
pitch_2019_copy['2019_actual_so'] = pitch_2019_copy.sort_values(['Team', 'Season'], ascending=False).groupby('Team')['SO'].shift()
pitch_2019_copy['2019_actual_era'] = pitch_2019_copy.sort_values(['Team', 'Season'], ascending=False).groupby('Team')['ERA'].shift()
pitch_2019_copy['2019_actual_h/9'] = pitch_2019_copy.sort_values(['Team', 'Season'], ascending=False).groupby('Team')['H/9'].shift()
pitch_2019_copy['2019_actual_hr/9'] = pitch_2019_copy.sort_values(['Team', 'Season'], ascending=False).groupby('Team')['HR/9'].shift()
pitch_2019_copy['2019_actual_ev'] = pitch_2019_copy.sort_values(['Team', 'Season'], ascending=False).groupby('Team')['EV'].shift()


pitch_2019_copy = pitch_2019_copy.loc[pitch_2019_copy['2019_actual_so'].notnull()]

pitch_2019_copy


In [ ]:
x = pitch_2019_copy[['G', 'H', 'SO', 'ERA', 'H/9', 'HR/9','EV' ]].values
y = pitch_2019_copy[['2019_actual_g', '2019_actual_h','2019_actual_so', '2019_actual_era', '2019_actual_h/9', '2019_actual_hr/9','2019_actual_ev']].values



In [ ]:
y_pred = lr.predict(x)
print('Mean of Stats:', pitch_2019_copy.mean())
print('Mean absolute error:', mean_absolute_error(y_pred, y))

In [ ]:
pitch_2019_copy[['Predicted_G', 'Predicted_H', 'Predicted_SO', 'Predicted_ERA', 'Predicted_H/9', 'Predicted_HR/9','Predicted_EV']] = y_pred

pitch_2019_copy['Season'] = 2019

#pitch_2019_copy = pitch_2019_copy.rename(columns={'2019_actual_g':'Actual_G'}

#use sort_values to find the top predicted hits
pitch_2019_copy = pitch_2019_copy.loc[:,['Season','Team','2019_actual_g', '2019_actual_h', '2019_actual_so', '2019_actual_era', '2019_actual_h/9' , '2019_actual_hr/9', 'Predicted_G', 'Predicted_H', 'Predicted_SO', 'Predicted_ERA', 'Predicted_H/9', 'Predicted_HR/9','Predicted_EV' ]]



In [ ]:
pitch_2019_copy

In [ ]:
predicted_2020_stats = pitch_2019_copy.loc[:,['Team', 'Predicted_G', 'Predicted_H', 'Predicted_SO', 'Predicted_ERA', 'Predicted_H/9', 'Predicted_HR/9','Predicted_EV']]

predicted_2020_stats


In [ ]:
predicted_2020_stats.sum()

Team              HOULADCHCARIMILTBRATLBOSCLENYYOAKSTLSFGPITWSNNYMSEAPHILAACOLSDPMINDETCINMIACHWTORTEXKCRBAL
Predicted_G      21233.51125                                                                                
Predicted_H      41469.96162                                                                                
Predicted_SO     42057.34419                                                                                
Predicted_ERA    130.76561                                                                                  
Predicted_H/9    258.18794                                                                                  
Predicted_HR/9   39.29857                                                                                   
Predicted_EV     2652.15757                                                                                 
dtype: object